## 👋🏼 Get Started

This example shows how to **reuse pre-trained models** with ZhiJian.

- **Why** ZhiJian?

    - **Effortlessly** and **swiftly** customize and reuse *any* pre-trained model on *any* dataset you want.

- **All in just 10 minutes**

    - 1 min to install **ZhiJian**
    - 2 mins to select the dataset
    - 3 mins to customize the model structure
    - 4 mins to deploy training and test process


🚀 **Let's get started!**

#### Install ZhiJian

In [5]:
!pip install zhijian

In [6]:
import zhijian
print(zhijian.__version__)

0.0.1


#### Select Dataset

ZhiJian provides the loading interface for *19 datasets* spanning several domains, including general objects, animals and plants, food and daily necessities, medicine, and remote sensing. These datasets cover the [`VTAB` benchmark](https://google-research.github.io/task_adaptation/).

Customize your own dataset, please see [here](TODO).

+ For better prompting, we first create a tool function that guides the input:

In [7]:
def select_from_input(prompt_for_select, valid_selections):
    selections2print = '\n\t'.join([f'[{idx + 1}] {i}' for idx, i in enumerate(valid_selections)])
    while True:
        selected = input(f"Please input a {prompt_for_select}, type 'help' to show the options: ")

        if selected == 'help':
            print(f"Available {prompt_for_select}(s):\n\t{selections2print}")
        elif selected.isdigit() and int(selected) >= 1 and int(selected) <= len(valid_selections):
            selected = valid_selections[int(selected) - 1]
            break
        elif selected in valid_selections:
            break
        else:
            print("Sorry, input not support.")
            print(f"Available {prompt_for_select}(s):\n\t{selections2print}")

    print(f"Your selection: {selected}")
    return selected

+ Now, run the following code block, input and select the target dataset and corresponding directory to be deployed:

In [8]:
available_datasets = [
    'VTAB-1k.CIFAR-100', 'VTAB-1k.CLEVR-Count', 'VTAB-1k.CLEVR-Distance', 'VTAB-1k.Caltech101', 'VTAB-1k.DTD',
    'VTAB-1k.Diabetic-Retinopathy', 'VTAB-1k.Dmlab', 'VTAB-1k.EuroSAT', 'VTAB-1k.KITTI', 'VTAB-1k.Oxford-Flowers-102',
    'VTAB-1k.Oxford-IIIT-Pet', 'VTAB-1k.PatchCamelyon', 'VTAB-1k.RESISC45', 'VTAB-1k.SUN397', 'VTAB-1k.SVHN',
    'VTAB-1k.dSprites-Location', 'VTAB-1k.dSprites-Orientation', 'VTAB-1k.smallNORB-Azimuth', 'VTAB-1k.smallNORB-Elevation'
] # dataset options.
dataset = select_from_input('dataset', available_datasets)  # user input about dataset

Available dataset(s):
	[1] VTAB-1k.CIFAR-100
	[2] VTAB-1k.CLEVR-Count
	[3] VTAB-1k.CLEVR-Distance
	[4] VTAB-1k.Caltech101
	[5] VTAB-1k.DTD
	[6] VTAB-1k.Diabetic-Retinopathy
	[7] VTAB-1k.Dmlab
	[8] VTAB-1k.EuroSAT
	[9] VTAB-1k.KITTI
	[10] VTAB-1k.Oxford-Flowers-102
	[11] VTAB-1k.Oxford-IIIT-Pet
	[12] VTAB-1k.PatchCamelyon
	[13] VTAB-1k.RESISC45
	[14] VTAB-1k.SUN397
	[15] VTAB-1k.SVHN
	[16] VTAB-1k.dSprites-Location
	[17] VTAB-1k.dSprites-Orientation
	[18] VTAB-1k.smallNORB-Azimuth
	[19] VTAB-1k.smallNORB-Elevation
Your selection: VTAB-1k.CIFAR-100


In [9]:
dataset_dir = input(f"Please input your dataset directory: ")   # user input about dataset directory
print(f"Your dataset directory: {dataset_dir}")

Your dataset directory: /data/zhangyk/data/zhijian


#### Customize Pre-trained Model

Seamlessly modify the structure is possible. **ZhiJian** welcomes any base model and any additional modifications. The base part supports:
  - 🤗 **Hugging Face** series — [PyTorch Image Models (timm)](https://github.com/huggingface/pytorch-image-models), [Transformers](https://github.com/huggingface/transformers), **PyTorch** series — [Torchvision](https://pytorch.org/vision/stable/models.html), and **OpenAI** series — [CLIP](https://github.com/openai/CLIP).
  - Other popular projects, *e.g.*, [vit-pytorch](https://github.com/lucidrains/vit-pytorch) (stars [14k](https://github.com/lucidrains/vit-pytorch/stargazers)) and **any custom** architecture.
  - **Large Language Model**, including [baichuan](https://huggingface.co/baichuan-inc/baichuan-7B) (*7B*), [LLaMA](https://github.com/facebookresearch/llama) (*7B/13B*), and [BLOOM](https://huggingface.co/bigscience/bloom) (*560M/1.1B/1.7B/3B/7.1B*).

**ZhiJian** also includes assembling additional tuning structures, similar to building *LEGO* bricks. For more detailed customization of each part, please see [here](TODO).

Adapt the `Vision Transformer` structure just requires **1~3** lines of code.

+ Now, run the following code block, input and select the model architecture:

In [10]:
available_example_models = {
    'timm.vit_base_patch16_224_in21k': {
        'LoRA': '(LoRA.adapt): ...->(blocks[0:12].attn.qkv){inout1}->...',
        'Adapter': '(Adapter.adapt): ...->(blocks[0:12].drop_path1){inout1}->...',
        'Convpass': ('(Convpass.adapt): ...->(blocks[0:12].norm1){in1}->(blocks[0:11].drop_path1){in2}->...,' # follow the next line
                     '(Convpass.adapt): ...->{in1}(blocks[0:11].norm2)->(blocks[0:12].drop_path2){in2}->...'),
        'None': None
    }
} # model options, Dict(model name: Dict(add-in structure name: add-in blitz configuration)).

model = select_from_input('model', list(available_example_models.keys())) # user input about model

Available model(s):
	[1] timm.vit_base_patch16_224_in21k
Your selection: timm.vit_base_patch16_224_in21k


+ Next, run the following code block, input and select the additional add-in structure for parameter-efficient transfer:

In [11]:
availables   = available_example_models[model]
config_blitz = availables[select_from_input('add-in structure', list(availables.keys()))] # user input about add-in structure

Available add-in structure(s):
	[1] LoRA
	[2] Adapter
	[3] Convpass
	[4] None
Your selection: LoRA


#### Deploy the Training and Test Process

**ZhiJian** enables customization of the updated portion using `args.reuse_key`, *such as* assigning `blocks[6:8]` to only tune `model.blocks[6]` to `model.blocks[8]` and their sub-modules.

**ZhiJian** also supports diverse training and testing methodologies, including **knowledge distillation** with teacher model supervision, **regularization** under model initialization constraints, **model merging** for comprehensive adaptation settings, and so on.

+ Now, run the following code block, input and select the parameters to fine-tune (the rest are frozen)

In [13]:
available_example_reuse_modules = {
    'timm.vit_base_patch16_224_in21k': {
        'add-ins and linear layer': 'addin,head,fc_norm',
        'add-ins and the last block and the linear layer (Partial-1)': 'addin,blocks[11],head,fc_norm',
        'add-ins and the last two blocks and the linear layer (Partial-2)': 'addin,blocks[10:12],head,fc_norm',
        'add-ins and the last four blocks and the linear layer (Partial-4)': 'addin,blocks[8:12],head,fc_norm',
    }
}

availables       = available_example_reuse_modules[model]
reuse_keys_blitz = availables[select_from_input('reuse modules', list(availables.keys()))] # user input about reuse modules

Available reuse modules(s):
	[1] add-ins and linear layer
	[2] add-ins and the last block and the linear layer (Partial-1)
	[3] add-ins and the last two blocks and the linear layer (Partial-2)
	[4] add-ins and the last four blocks and the linear layer (Partial-4)
Your selection: add-ins and linear layer


+ Taking *finetune* mode as an example, next, we configure the parameters

  For the rest of the training configuration with more customization options, such as knowledge distillation, regular constraints and model merging, please see [here](TODO).

In [14]:
from zhijian.trainers.base import prepare_args
from zhijian.models.utils import pprint, dict2args
training_mode = 'finetune'
args = dict2args({
    'log_url': 'your/log/directory',        # log directory
    'dataset': dataset,                     # dataset
    'dataset_dir': dataset_dir,             # dataset directory
    'model': model,                         # backbone network
    'config_blitz': config_blitz,           # addin blitz configuration
    'training_mode': training_mode,         # training mode
    'reuse_keys_blitz': reuse_keys_blitz,   # reuse keys blitz configuration
    'optimizer': 'adam',                    # optimizer
    'batch_size': 64,                       # batch size
    'num_workers': 8,                       # num workers
    'max_epoch': 5,                         # max epoch
    'eta_min': 0,                           # eta_min of CosineAnnealingLR
    'lr': 1e-3,                             # learning rate
    'wd': 5e-5,                             # weight decay
    'gpu': '0',                             # gpu id
    'seed': 0,                              # random seed
    'verbose': True,                        # control the verbosity of the output
    'only_do_test': False                   # test flag
})

args = prepare_args(args, update_default=True)
pprint(vars(args))

2023-07-18 15:17:49.411113: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-18 15:17:49.595826: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-18 15:17:50.315077: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/zhangyk/miniconda3/lib
2023-07-18 15:17:50.315154: W tensorflow/compiler/xla/stre

{'aa': None,
 'addins': [{'hook': [['adapt', 'post']],
             'location': [['blocks', 0, 'attn', 'qkv']],
             'name': 'LoRA'},
            {'hook': [['adapt', 'post']],
             'location': [['blocks', 1, 'attn', 'qkv']],
             'name': 'LoRA'},
            {'hook': [['adapt', 'post']],
             'location': [['blocks', 2, 'attn', 'qkv']],
             'name': 'LoRA'},
            {'hook': [['adapt', 'post']],
             'location': [['blocks', 3, 'attn', 'qkv']],
             'name': 'LoRA'},
            {'hook': [['adapt', 'post']],
             'location': [['blocks', 4, 'attn', 'qkv']],
             'name': 'LoRA'},
            {'hook': [['adapt', 'post']],
             'location': [['blocks', 5, 'attn', 'qkv']],
             'name': 'LoRA'},
            {'hook': [['adapt', 'post']],
             'location': [['blocks', 6, 'attn', 'qkv']],
             'name': 'LoRA'},
            {'hook': [['adapt', 'post']],
             'location': [['blocks', 7, 'a

+ Next, run the following code block to configure the GPU:

In [15]:
import torch
import os
assert torch.cuda.is_available()
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu
torch.cuda.set_device(int(args.gpu))

+ Run the following to get the pre-trained model, which includes the additional add-in modules that have been accessed:

In [16]:
from zhijian.trainers.finetune import get_model
model, model_args, device = get_model(args)

/home/zhangyk/miniconda3/envs/zykycy/lib/python3.9/site-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name vit_base_patch16_224_in21k to current vit_base_patch16_224.augreg_in21k.
  model = create_fn(


+ Run the following to get the `dataloader`:

In [17]:
from zhijian.data.base import prepare_vision_dataloader
train_loader, val_loader, num_classes = prepare_vision_dataloader(args, model_args)

+ Run the following to prepare the optimizer, learning rate scheduler and loss function

In [18]:
import torch.nn as nn
import torch.optim as optim
optimizer = optim.Adam(
    model.parameters(),
    lr=args.lr,
    weight_decay=args.wd
)
lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    args.max_epoch,
    eta_min=args.eta_min
)
criterion = nn.CrossEntropyLoss()

+ Run the following to initialize the `trainer`, ready to **start training**:

In [19]:
from zhijian.trainers.base import prepare_trainer
trainer = prepare_trainer(
    args,
    model=model, model_args=model_args, device=device,
    train_loader=train_loader, val_loader=val_loader, num_classes=num_classes,
    optimizer=optimizer, lr_scheduler=lr_scheduler, criterion=criterion
)

Log level set to: INFO
Log files are recorded in: your/log/directory/0718-15-17-52-580
Trainable/total parameters of the model: 0.37M / 86.17M (0.43148%)


+ Run the following to train and test with **ZhiJian**

In [20]:
trainer.fit()
trainer.test()


      Epoch   GPU Mem.       Time       Loss         LR


        1/5      7.16G     0.3105      4.629      0.001: 100%|██████████| 16.0/16.0 [00:04<00:00, 3.66batch/s]



      Epoch   GPU Mem.       Time      Acc@1      Acc@5


        1/5      7.16G     0.1188      3.334      14.02: 100%|██████████| 157/157 [00:18<00:00, 8.35batch/s] 
***   Best results: [Acc@1: 3.3339968152866244], [Acc@5: 14.022691082802547]



      Epoch   GPU Mem.       Time       Loss         LR


...


      Epoch   GPU Mem.       Time      Acc@1      Acc@5


        5/5      7.16G      0.119      5.723      19.39: 100%|██████████| 157/157 [00:18<00:00, 8.33batch/s] 
***   Best results: [Acc@1: 5.722531847133758], [Acc@5: 19.386942675159236]



      Epoch   GPU Mem.       Time      Acc@1      Acc@5


        1/5      7.16G     0.1192      5.723      19.39: 100%|██████████| 157/157 [00:18<00:00, 8.30batch/s] 
***   Best results: [Acc@1: 5.722531847133758], [Acc@5: 19.386942675159236]


(5.722531847133758, 19.386942675159236)